In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from radiomics import featureextractor, setVerbosity
setVerbosity(50)

import SimpleITK as sitk

In [ ]:
# filepath setup
DATA_SOURCE = "TCIA"
DATASET_NAME = "HEAD-NECK-RADIOMICS-HN1"
DATA_DIR = Path("../data/procdata", f"{DATA_SOURCE}_{DATASET_NAME}")
MIT_INDEX_DIR = "mit_" + DATASET_NAME


data_index_file = Path(DATA_DIR, MIT_INDEX_DIR, f"{MIT_INDEX_DIR}_index.csv")
pyrad_output_path = Path(DATA_DIR, "pyradiomics_features")
pyrad_output_path.mkdir(parents = True, exist_ok=True)

In [ ]:
# Construct file path lists for images and masks
image_files = sorted((DATA_DIR / MIT_INDEX_DIR).rglob(pattern="*/CT*/CT.nii.gz"))
mask_files = sorted((DATA_DIR / MIT_INDEX_DIR).rglob(pattern="*/RT*/*.nii.gz"))

# Get list of sample IDs from top of data directory
unique_sample_ids = [sample_dir.name for sample_dir in sorted((DATA_DIR / MIT_INDEX_DIR).glob(pattern="*/"))]

# Construct dataframe to iterate over
pyrad_dataset_df = pd.DataFrame(data = {'ID': unique_sample_ids, 'Image': image_files, 'Mask': mask_files})

In [ ]:
# Save out the dataframe as a csv for pyradiomics to use for feature extraction
pyrad_dataset_df.to_csv(pyrad_output_path / f"pyrad_{DATASET_NAME}_index.csv", index=False)

### Make pyradiomics dataframe from MIT output
Disabled until index file issues are resolved

In [ ]:
# Read in index file output by imgtools autopipeline
# dataset_index = pd.read_csv(data_index_file)

# List of image IDs to keep - which masks to use with the image (CT)
# keep_image_ids = ['020_00', '030_00', '040_00', '050_00', 'acryllic_00', 'cork_00', 'dcork_00', 'resin_00', 'rubber_00', 'wood_00', 'CT']

# Select out just the rows from the dataset index with the selected IDs
# dataset_index = dataset_index[dataset_index['ImageID'].isin(keep_image_ids)]

# Setting up the pyradiomics input file 
# Initialize an empty list to store the image, mask pairs as lists
# pyrad_dataset_list = []
# # Get the unique samples from the dataset index
# # unique_sample_ids = np.sort(dataset_index['SampleID'].unique())

# relative_path_prefix = Path(DATA_DIR, MIT_INDEX_DIR)

# for id in unique_sample_ids:
#     # Get all the index rows associated with this sample
#     sample_rows = dataset_index[dataset_index['SampleID'] == id]
#     try:
#         # Get the image row - this currently assumes there's only one CT per sample
#         image_row = sample_rows[sample_rows['Modality'] == 'CT']

#         # Get the path to the image file as a list to remove the index, metadata, etc. from the dataframe
#         image_path = image_row['filepath'].to_list()
#         image_path = [relative_path_prefix / image_path[0]]
#     except IndexError as e:
#         print(f"Missing CT for sample {id}")
#         continue

#     try:
#         # Iterate over the mask files associated with this sample image
#         for mask_row in sample_rows[sample_rows['Modality'] == 'RTSTRUCT'].itertuples(index=False):
#             # Make a list of the sampleID, image and mask paths and then append this to the dataset list
#             pyrad_dataset_list.append([id] + image_path + [relative_path_prefix / mask_row[0]])
#     except IndexError as e:
#         print(f"Missing RTSTRUCT for sample {id}")
#         continue

# # Convert the list of image + mask pair lists to a dataframe for saving
# pyrad_dataset_df = pd.DataFrame(data=pyrad_dataset_list, columns=['ID', 'Image', 'Mask'])

# Pyradiomics Feature Extraction

In [ ]:
params = "../src/readii/data/shape_only_pyradiomics.yaml"
extractor = featureextractor.RadiomicsFeatureExtractor(params)

result_list = []

for sample in pyrad_dataset_df.itertuples(index=False):
    print(f"Processing {sample[0]}")
    sample_image = sitk.ReadImage(sample[1])
    sample_mask = sitk.ReadImage(sample[2])

    try:
        sample_feature_vector = pd.Series(extractor.execute(sample_image, sample_mask))
    except Exception as e:
        print("Feature extraction failed: ", e)

    sample_data = [sample[0], sample[1].name, sample[2].name, sample[1], sample[2]] + sample_feature_vector.to_list()

    result_list.append(sample_data)

col_names = pyrad_dataset_df.columns.to_list() + ["Image_path", "Mask_path"] + sample_feature_vector.index.to_list()
results_df = pd.DataFrame(data=result_list, columns=col_names)

results_df.to_csv(pyrad_output_path / f"pyrad_{DATASET_NAME}_features.csv", index=False)